In [2]:
from pyspark.sql.session import SparkSession

from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .enableHiveSupport() \
    .getOrCreate()
# df = spark.read.option("header", True).csv("/Users/FelixLi/SourceCode/python/functions/output/amex/data.csv")
# df.printSchema()



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/07 15:17:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/07 15:17:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [1]:
!pip install more-itertools

In [3]:
from pyspark.accumulators import AccumulatorParam
class ListAccumulatorParam(AccumulatorParam):
    def zero(self, initialValue=[]):
        return []
 
    def addInPlace(self, s1, s2):
        s1 += [s2]
        return s1

partition_list = spark.sparkContext.accumulator([], ListAccumulatorParam())


In [12]:
%%time
table = spark.table("cs_amex.cs_amex_pageviews")
from more_itertools import take
table.select('dt').foreachPartition(lambda rdd: partition_list.add(take(1,rdd)[0]['dt']))
partition_list
print(len(set([item[0] for item in  partition_list.value])))
pass

37
CPU times: user 201 ms, sys: 37.2 ms, total: 238 ms
Wall time: 1min 52s


In [11]:
%%time
table = spark.table("cs_amex.cs_amex_pageviews_parquet")
from more_itertools import take
table.select('dt').foreachPartition(lambda rdd: partition_list.add(take(1,rdd)[0]['dt']))
partition_list
print(len(set([item[0] for item in  partition_list.value])))
pass

37
CPU times: user 53.3 ms, sys: 16.2 ms, total: 69.5 ms
Wall time: 9.56 s


In [4]:
%%time
table = spark.table("cs_amex.cs_amex_pageviews")
from more_itertools import take
partition_list = []
table.select('dt').foreachPartition(lambda rdd: partition_list.append(take(1,rdd)[0]['dt']))

22/11/07 13:42:54 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


CPU times: user 185 ms, sys: 59.4 ms, total: 244 ms
Wall time: 1min 53s


In [ ]:
%%time
table.select('dt').foreachPartition(lambda rdd: partition_list.add(take(1,rdd)[0]['dt']))
partitionList

In [5]:
import CustomAccumulators as ca
acc = spark.sparkContext.accumulator(dict(), ca.DictAccumulator(ca.DictAccumulatorMethod.LIST))
acc_manager = AccumulatorManager(acc)

ModuleNotFoundError: No module named 'CustomAccumulators'

In [7]:
from pyspark.accumulators import Accumulator

In [ ]:
table.foreachPartition(lambda rdd: print(rdd.select('dt').collect(1)))

In [ ]:
spark.sql( "SHOW PARTITIONS table_tmp")

In [ ]:
table

In [ ]:
%%time
table.select('view_number').filter('dt = "20220701" ').count()

In [ ]:
%%time
spark.sql("select view_number from cs_amex.cs_amex_pageviews where dt='20220701'").count()
 

In [ ]:
%%time
spark.sql("select view_number from cs_amex.cs_amex_pageviews").filter('dt = "20220701" ').count()

In [ ]:
%%time
table.select("dt").distinct().toPandas()

In [8]:
from pyspark.sql.dataframe import DataFrame as sparkDF
from more_itertools import take
from pyspark.accumulators import AccumulatorParam
from pyspark.sql.session import SparkSession

from pyspark.accumulators import AccumulatorParam
class ListAccumulatorParam(AccumulatorParam):
    def zero(self, initialValue=[]):
        return []

    def addInPlace(self, s1, s2):
        s1 += [s2]
        return s1


def get_partition_list(table: sparkDF, spark: SparkSession):
    partition_list = spark.sparkContext.accumulator([], ListAccumulatorParam())
    table.select('dt').foreachPartition(lambda rdd: partition_list.add(take(1,rdd)[0]['dt']))
    return set([item[0] for item in  partition_list.value])


spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .enableHiveSupport() \
    .getOrCreate()



In [9]:
%%time

table = spark.table("cs_amex.cs_amex_pageviews_parquet")
get_partition_list(table, spark)
 

CPU times: user 60.4 ms, sys: 16.6 ms, total: 77 ms
Wall time: 10.3 s


{'20220410',
 '20220701',
 '20220702',
 '20220703',
 '20220704',
 '20220705',
 '20220706',
 '20220707',
 '20220708',
 '20220709',
 '20220710',
 '20220711',
 '20220712',
 '20220713',
 '20220714',
 '20220715',
 '20220716',
 '20220717',
 '20220718',
 '20220719',
 '20220720',
 '20220721',
 '20220722',
 '20220723',
 '20220724',
 '20220725',
 '20220726',
 '20220727',
 '20220728',
 '20220729',
 '20220730',
 '20220731',
 '20220801',
 '20220808',
 '20220815',
 '20220822',
 '20220829'}

In [ ]:
%%time

table = spark.table("cs_amex.cs_amex_pageviews")
get_partition_list(table, spark)
 